## Session #1: Sampling from a Gaussian Process

#### Welcome to the first coding session of the workshop! 

In this session we will get a first intuition for for the definition of a Gaussian process. 

A Gaussian process can be seen as an infinite dimensional Gaussian distribiution - a distribution over functions. Like any other Gaussian distribution it is fully defined by a mean $m(x)$ and a covariance function $k(x, x')$: 
\begin{align}
f(x) \sim \mathcal{GP}( m(x), k(x, x'))
\end{align}

The mean $m(x)$ just gives the mean of the random variable at $x$ for any $x \in \mathbb{R}$. The covariance function $k(x, x')$ gives the covariance of any two random variables $x$ and $x'$. It determines the autocorrelation length or the 'smoothness' of the functions defined by the Gaussian process. 

## Your task: 

Your task is to 

a) define a Gaussian process, to 

b) draw function samples from it and to 

c) look at the distribution one or several random variables over samples. 

### a) Defining a Gaussian Process: 

To keep things simple, let us define the mean function to be just zero: $m(x) = 0$. Then, the only thing we still need is a covariance function. For a start we take the squared exponential covariance function: 

\begin{align}
k(x, x') = \exp(- 0.5 |x - x'|^2)
\end{align}

I will provide a function that takes any two arrays $x$ and $x'$ and returns the corresponding covariance matrix $K$. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('seaborn-deep')

In [ ]:
def calculate_covariance_matrix(x_p, x_q): 

    # for a convenient computation, we need a two dimensional array
    if x_p.ndim < 2: 
        x_p = x_p.reshape(-1 ,1)
    if x_q.ndim < 2: 
        x_q = x_q.reshape(-1, 1)
    
    # calculate the squared distance: x^2 - 2xy + y^2
    square_dist = np.sum(x_p ** 2, 1).reshape(-1, 1) + np.sum(x_q ** 2, 1) - 2 * np.dot(x_p, x_q.T)
    
    # return the exponential of the squared distance
    return np.exp(-0.5 * square_dist)

In [ ]:
# define some values x over a range. Make sure to samples densely for better visualization as a 'function'. 


# calculate the corresponding covariance matrix k(x, x')


# test: plot the matrix using plt.imshow. it should be bright on the first diagonal and exponentially decaying. 



### b) Sampling from a multivariat Gaussian distribution

Samples from the Gaussian process are infinite objects in theory, but we will of course sample finite objects, i.e., we sample a long vector and treat it as a 'function'. So effectively, we want to sample from a multivariat Gaussian distribution. How do we do that? 

There is no simple thing like np.random.normal for a d-dimensional case. But I will provide a function that gives a $n$ samples from a $d$-dimensional Gaussian distribution with arbitrary mean and covariance functions. For details, see below. 

In [ ]:
def sample_from_multivariat_gaussian(m, k, n_dimensions, n_samples): 
    """
    Draw samples from an arbitrary Gaussian distribution with mean m and covariance k. 
    
    The dimensions of m and k have to correspond, e.g., 
    """
    # calculate cholesky decomposition. add a bit of noise for numerical stability
    L = np.linalg.cholesky(k + 1e-6 * np.random.rand() * np.eye(n_dimensions))

    # draw samples from a 1-D Gaussian distribution 
    u = np.random.normal(size=(n_dimensions, n_samples))

    # use the cholesky decomposition to provide these samples with 
    # the covariance structure of the multidimensional Gaussian
    return m + L.dot(u)

In [ ]:
# draw some samples, e.g., n_samples=1000. 
# Use m=0 and the covariance matrix you defined above. What is the number of dimensions then? 



What is returned is a (n_dimensions, n_samples) shaped matrix. It contains your sampled functions! In which dimension of the matrix? 

In [ ]:
# plot some of the sampled 'functions' 



### Nice! You have just sampled functions from an infinite dimensional Gaussian distribution! 

### A Gaussian process is a collection of random variables, any finite number of which have a joint Gaussian distribution 

Please double check! Plot the distrubtion over samples at a single point $x$. 

In [ ]:
# to see the effect make sure that you esed a lot of samples in the GP above, e.g, at least n_samples=1000. 
# plot a histogram of the samples at a single point x



Now plot the join distribution of two points $x_1$ and $x_2$. 

In [ ]:
# you can use np.histrogram2d to get a matrix histogram from two vectors


# and then you could use plt.imshow to plot this histogram matrix



### Well done! 

#### We have defined a Gaussian process as a collection of random variables and sampled from this collection. 

#### We demonstrated that any subset of this collection is a Gaussian distribution itself. 

### Sampling using the Cholesky decomposition

To generate samples $x \sim \mathcal{N}(m, K)$ with arbitrary mean **m** and covariance K using a Gaussian scalar generator (e.g., np.random.normal) we proceed as follows: 

We compute the Cholesky decomposition $L$ of the positive definite covariance matrix $K$, $K=LL^T$, where $L$ is a lower triangular matrix. Then generate $u \sim \mathcal{N}(0, I)$ Gaussian scalar samples. Compute $x = m + Lu$, which has the desired distribution with mean $m$ and covariance $L \mathcal{E}[uu^T]L^T = LL^T = K$. 

In practice it may be necessary to add a small multiple of the identity matrix $\epsilon I$ to the covariance matrix for numerical reasons. This is because the eigenvalues of the matrix $K$ can decay very rapidly (see section 4.3.1 for 
a closely related analytical result) and without this stabilization the Cholesky decomposition fails.